In [2]:
import os
from os import environ
import pandas as pd
import numpy as np
import geopandas as gpd
from sodapy import Socrata
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

In [3]:
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.kernel_ridge import KernelRidge

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import os.path

In [5]:
path = '/Users/Albert/Jupyter/Tutorial/pluto_21v1.csv'

In [6]:
df = pd.read_csv(path, low_memory=False, 
                 dtype={'block': str, 'lot': str,
                        'cd': str, 'ct2010': str,
                        'cb2010': str, 'schooldist': str,
                        'council': str, 'zipcode': str,
                        'landuse': str, 'bbl': str})
df['bbl'] = df['bbl'].str.replace('.00000000', '')

In [7]:
print(df.shape)
df.head(3)

(858774, 90)


,borough,block,lot,cd,ct2010,cb2010,schooldist,council,zipcode,firecomp,...,appbbl,appdate,plutomapid,firm07_flag,pfirm15_flag,version,dcpedited,latitude,longitude,notes
0,SI,1701,96,501,303.02,4004,31,49,10303,E166,...,5.017010e+09,01/02/1992,1,NaN,NaN,21v1,NaN,40.624031,-74.155358,NaN
1,MN,16,7502,101,317.04,2000,02,1,10280,E010,...,1.000162e+09,08/25/1988,1,1.0,1.0,21v1,NaN,40.710107,-74.017570,NaN
2,MN,348,46,103,14.02,2001,01,1,10002,L018,...,NaN,NaN,1,NaN,NaN,21v1,NaN,40.718566,-73.984816,NaN


In [9]:
path = "/Users/Albert/Jupyter/Tutorial/Building Footprints.geojson"
bldgs = gpd.read_file(path, dtype={'shape_area': float, 'heightroof': float,
                                  'cnstrct_yr': int, 'groundelev': int,
                                  'shape_len': float})

In [10]:
bldgs.rename(columns={'base_bbl': 'bbl'}, inplace=True)

In [11]:
print(bldgs.shape)
bldgs.head(3)

(1139213, 15)


,name,bbl,shape_area,heightroof,mpluto_bbl,cnstrct_yr,lststatype,feat_code,groundelev,geomsource,bin,lstmoddate,doitt_id,shape_len,geometry
0,None,3044520815,854.66243317866,21.60850812,3044520815,2009,Constructed,2100,18,Photogramm,3394646,2017-08-22T00:00:00+00:00,1212853,125.0797955584,"MULTIPOLYGON (((-73.87130 40.65717, -73.87136 ..."
1,None,4030640041,217.59424346169,10.36,4030640041,1930,Constructed,5110,122,Photogramm,4548330,2017-08-17T00:00:00+00:00,1226227,60.22585821856,"MULTIPOLYGON (((-73.87671 40.71425, -73.87677 ..."
2,None,4139430001,946.42747637737,29.81157033,4139430001,1960,Constructed,2100,10,Photogramm,4460479,2017-08-22T00:00:00+00:00,581946,123.14194057237,"MULTIPOLYGON (((-73.85195 40.66235, -73.85195 ..."


In [12]:
enpath = '/Users/Albert/Jupyter/Final/bldg_energy.csv'
energy = gpd.read_file(enpath)

In [13]:
energy.rename(columns={'bbl_10_digits': 'bbl',
                            'nyc_building_identification': 'bin'}, 
                   inplace=True)

In [14]:
bcols = ['bbl', 'bin','total_ghg_emissions_metric', 'direct_ghg_emissions_metric','electricity_use_grid_purchase', 'natural_gas_use_kbtu', 'occupancy']
temp = energy[bcols].merge(df, on='bbl')

In [18]:
cols = ['easements', 'lotarea', 'bldgarea', 'landuse',
        'comarea', 'resarea', 'officearea', 'retailarea',
        'garagearea', 'strgearea', 'factryarea', 'otherarea',
        'areasource', 'numbldgs', 'numfloors', 'unitsres',
        'unitstotal', 'lotfront', 'lotdepth', 'bldgfront',
        'bldgdepth','yearbuilt', 'yearalter1', 'yearalter2',
        'borough','assessland','assesstot']

In [19]:
temp = temp[cols+bcols]

In [20]:
bfcols = ['shape_area', 'heightroof', 'groundelev',
          'bin', 'shape_len', 'geometry']

In [21]:
temp = temp.merge(bldgs[bfcols], on=['bin'])

In [22]:
temp.drop_duplicates(inplace=True)

In [23]:
temp.fillna({'otherarea':0,'factryarea':0,'strgearea':0,'garagearea':0,
             'retailarea':0,'officearea':0,'resarea':0,'comarea':0},
            inplace=True)

In [24]:
temp.dropna(inplace=True)

In [25]:
temp.shape

(24248, 39)

In [26]:
temp.head(3)



,easements,lotarea,bldgarea,landuse,comarea,resarea,officearea,retailarea,garagearea,strgearea,...,total_ghg_emissions_metric,direct_ghg_emissions_metric,electricity_use_grid_purchase,natural_gas_use_kbtu,occupancy,shape_area,heightroof,groundelev,shape_len,geometry
0,0.0,33639.0,734668.0,05,734668.0,0.0,686025.0,11510.0,0.0,0.0,...,2747.2,0,25517772.4,,55,24877.9527614759,511.16,61,702.8747521918,"MULTIPOLYGON (((-73.98352 40.75669, -73.98374 ..."
1,0.0,18993.0,380000.0,05,380000.0,0.0,360000.0,20000.0,0.0,0.0,...,2752.2,0,17555048.6,,100,18317.2285359523,342.32,35,597.31909526454,"MULTIPOLYGON (((-73.97413 40.75047, -73.97424 ..."
2,0.0,26100.0,646995.0,05,565001.0,0.0,565000.0,1.0,0.0,0.0,...,3038.1,50.5,25434413.5,950954.5,95,25382.2911506843,370.67,52,713.51271530925,"MULTIPOLYGON (((-73.98279 40.75803, -73.98289 ..."


In Manhattan

In [27]:
temp = temp[(temp['borough'] == 'MN')]

In [28]:
temp.shape

(9398, 39)

In [29]:

def standalone_check(x):
    temp = bldgs.loc[bldgs['geometry'].touches(x)]
    check = temp.shape[0] == 0
    return check

In [30]:
temp['standalone'] = temp['geometry'].apply(lambda x: standalone_check(x))

In [31]:
temp.shape

(9398, 40)

In [111]:
#archive = temp
#archive['standalonenum']

In [36]:
#temp['standalone']

In [37]:
temp['standalonenum'] = temp['standalone'].astype(int)

In [53]:
temp['standalonenum']
temp.shape


(8869, 41)

In [51]:
temp = archive

In [47]:
temp.drop_duplicates(subset='bin', inplace=True)

In [48]:
temp.fillna({'otherarea':0,'factryarea':0,'strgearea':0,'garagearea':0,
             'retailarea':0,'officearea':0,'resarea':0,'comarea':0},
            inplace=True)

In [49]:
temp.dropna(inplace=True)

In [199]:
archive.shape

(8869, 41)

In [55]:
gdf = gpd.GeoDataFrame(temp, geometry='geometry', crs='EPSG: 4326')

In [58]:
test = gpd.sjoin(tracts, gdf, op='intersects')

In [59]:
test.head()


,region_id,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,direct_ghg_emissions_metric,electricity_use_grid_purchase,natural_gas_use_kbtu,occupancy,shape_area,heightroof,groundelev,shape_len,standalone,standalonenum
518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,78008,...,0,19441041.2,,100,19432.9821782565,285.47,50,667.41256539481,False,0
518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,78008,...,0,2693285.8,,100,5260.02988549211,262.29,49,307.91553975476,False,0
518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,78008,...,0,17680208.2,,95,13315.4579949596,274.09,46,485.42150448012,False,0
518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,78008,...,0,54631173.9,,100,34355.59479662791,584.86,51,775.46029794089,False,0
518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,78008,...,0,,,100,5203.43596274268,110.94,53,301.66951959363,False,0


In [60]:
test.shape

(8874, 55)

In [62]:
test.to_csv('bldgsinfoaloneMN.csv')

Export File


In [112]:
test.to_csv('bldgsinfoaloneMNR.csv')

read file

In [1]:
test = pd.read_csv('bldginfoMNR.csv')

NameError: name 'pd' is not defined

In [302]:
test.shape

(9405, 54)

Building Height

In [63]:
test['heightroof'] = test['heightroof'].astype(float)

In [64]:
height = test.groupby('region_id')['heightroof'].mean()


In [65]:
height.head(3)
height.shape

(285,)

Ground Elevation

In [66]:
test['groundelev'] = test['groundelev'].astype(float)

In [67]:
elev = test.groupby('region_id')['groundelev'].mean()

In [68]:
elev.shape

(285,)

In [69]:
elev.head(3)

region_id
36005022102    95.0
36005028900    11.0
36047093400    27.0
Name: groundelev, dtype: float64

landuse

In [70]:
test['landuse'] = test['landuse'].astype(str)

In [71]:
landtest = test.reset_index()

In [72]:
landtest.shape

(8874, 56)

In [73]:
#landtest.groupby('region_id').agg({'landuse':'count'})

In [74]:
#landtest[['region_id','landuse']].values.tolist()

In [75]:
region_list = landtest['region_id'].values.tolist()
region_list = list(dict.fromkeys(region_list))
land_list = ['01','02','03','04','05','06','07','08','09','10','11']
#land_dict = {'region_id','01','02','03','04','05','06','07','08','09','10','11'}

In [76]:
#a = landtest[landtest['region_id'].isin(land_list)].groupby('landuse')['region_id']

In [77]:
for x in land_list:
    name = 'ld' + x
    landtest.loc[landtest['landuse'] == x, name ] = landtest['lotarea']

In [78]:
landtest.shape
landtest.head(3)

,index,region_id,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,...,ld02,ld03,ld04,ld05,ld06,ld07,ld08,ld09,ld10,ld11
0,518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,...,NaN,NaN,NaN,20075.0,NaN,NaN,NaN,NaN,NaN,NaN
1,518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,...,NaN,NaN,NaN,5021.0,NaN,NaN,NaN,NaN,NaN,NaN
2,518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,...,NaN,NaN,NaN,13500.0,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
landuse = pd.DataFrame()
for x in land_list:
    name = 'ld' + x
    landuse[name] = landtest.groupby('region_id')[name].sum()

In [80]:
landuse.shape


(285, 11)

In [81]:
landuse.head()

,ld01,ld02,ld03,ld04,ld05,ld06,ld07,ld08,ld09,ld10,ld11
region_id,,,,,,,,,,,
36005022102,0.0,0.0,4983.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36005028900,0.0,0.0,0.0,0.0,0.0,185000.0,0.0,0.0,0.0,0.0,0.0
36047093400,0.0,0.0,7550.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36061000201,0.0,0.0,0.0,0.0,0.0,0.0,568800.0,178466.0,0.0,0.0,0.0
36061000202,0.0,0.0,352765.0,22750.0,34515.0,0.0,568800.0,71361.0,0.0,0.0,0.0


In [82]:
#landuse = landuse.set_index('region_id')
landpct = landuse.div(landuse.sum(axis=1), axis=0)

In [83]:
landpct.head()

,ld01,ld02,ld03,ld04,ld05,ld06,ld07,ld08,ld09,ld10,ld11
region_id,,,,,,,,,,,
36005022102,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
36005028900,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.0,0.0
36047093400,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
36061000201,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.761175,0.238825,0.0,0.0,0.0
36061000202,0.0,0.0,0.335906,0.021663,0.032865,0.0,0.541616,0.067950,0.0,0.0,0.0


standalone

In [84]:
standalone = pd.DataFrame()

In [85]:
test['standalone_area'] = test['standalonenum'] * test['lotarea']

In [86]:
standalone['alonepct'] = test.groupby('region_id')['standalone_area'].sum()/test.groupby('region_id')['lotarea'].sum()

In [89]:
standalone.head(3)

,alonepct
region_id,
36005022102,0.0
36005028900,1.0
36047093400,0.0


tract FAR

In [90]:
far = pd.DataFrame()

In [91]:
far['far'] = test.groupby('region_id')['bldgarea'].sum()/test.groupby('region_id')['lotarea'].sum()

In [92]:
far.head(3)

,far
region_id,
36005022102,8.244431
36005028900,1.712432
36047093400,6.580132


Volume

In [93]:
test['shape_area'] = test['shape_area'].astype(float)

In [94]:
test['volume'] = test['shape_area'] * test['heightroof']

In [95]:
vol = test.groupby('region_id')['volume'].sum()

In [96]:
vol.head()

region_id
36005022102    8.015595e+05
36005028900    4.531667e+06
36047093400    1.804140e+04
36061000201    1.029213e+07
36061000202    2.064477e+07
Name: volume, dtype: float64

Pct


In [97]:
test['resarea'] = test['resarea'].astype(float)
test['officearea'] = test['officearea'].astype(float)
test['retailarea'] = test['retailarea'].astype(float)
test['factryarea'] = test['factryarea'].astype(float)
test['strgearea'] = test['strgearea'].astype(float)

In [98]:
test.head(5)

,region_id,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,natural_gas_use_kbtu,occupancy,shape_area,heightroof,groundelev,shape_len,standalone,standalonenum,standalone_area,volume
518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,78008,...,,100,19432.982178,285.47,50.0,667.41256539481,False,0,0.0,5.547533e+06
518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,78008,...,,100,5260.029885,262.29,49.0,307.91553975476,False,0,0.0,1.379653e+06
518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,78008,...,,95,13315.457995,274.09,46.0,485.42150448012,False,0,0.0,3.649634e+06
518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,78008,...,,100,34355.594797,584.86,51.0,775.46029794089,False,0,0.0,2.009321e+07
518,36061011202,36,61,11202,36061011202,112.02,Census Tract 112.02,G5020,S,78008,...,,100,5203.435963,110.94,53.0,301.66951959363,False,0,0.0,5.772692e+05


In [99]:
pct = pd.DataFrame()

In [100]:
pct['respct'] = test.groupby('region_id')['resarea'].sum()/test.groupby('region_id')['bldgarea'].sum()
pct['offpct'] = test.groupby('region_id')['officearea'].sum()/test.groupby('region_id')['bldgarea'].sum()
pct['retpct'] = test.groupby('region_id')['retailarea'].sum()/test.groupby('region_id')['bldgarea'].sum()
pct['facpct'] = test.groupby('region_id')['factryarea'].sum()/test.groupby('region_id')['bldgarea'].sum()
pct['strpct'] = test.groupby('region_id')['strgearea'].sum()/test.groupby('region_id')['bldgarea'].sum()

In [101]:
pct.head(10)

,respct,offpct,retpct,facpct,strpct
region_id,,,,,
36005022102,1.000000,0.000000,0.000000,0.0,0.000000
36005028900,0.000000,0.000000,0.005682,0.0,0.994318
36047093400,1.000000,0.000000,0.000000,0.0,0.000000
36061000201,0.000000,0.000000,0.000000,0.0,0.000000
36061000202,0.655314,0.010778,0.023757,0.0,0.000000
36061000600,0.348652,0.002078,0.009456,0.0,0.000255
36061000700,0.261048,0.644106,0.026864,0.0,0.014710
36061000800,0.702616,0.023114,0.041920,0.0,0.121130
36061000900,0.036028,0.895791,0.018778,0.0,0.002063


In [102]:
pct.fillna({'respct':0, 'offpct':0, 'retpct':0, 'facpct':0, 'strpct':0}, inplace=True)

Assess Value

In [200]:
assland = pd.DataFrame()


In [201]:
assland['assessland'] = test.groupby('region_id')['assessland'].sum()/test.groupby('region_id')['lotarea'].sum()

In [202]:
assland['assesstot'] = test.groupby('region_id')['assesstot'].sum()/test.groupby('region_id')['lotarea'].sum()

In [204]:
assland.head()

,assessland,assesstot
region_id,,
36005022102,12.010837,353.732691
36005028900,13.378378,76.118108
36047093400,203.841060,440.642384
36061000201,19.926639,69.719150
36061000202,27.117210,93.806650


In [205]:
assland.shape

(285, 2)

Emission

In [117]:
test.fillna({'total_ghg_emissions_metric':0, 'direct_ghg_emissions_metric':0}, inplace=True)

In [118]:
test.dropna(inplace=True)

In [119]:
test.shape

(8874, 57)

In [127]:
import re

In [138]:
total_list = test['total_ghg_emissions_metric'].values.tolist()

In [139]:
direct_list = test['direct_ghg_emissions_metric'].values.tolist()

In [149]:
region = test['region_id'].values.tolist()

In [168]:
Lt = []
Ld = []
for x in total_list:
    try:
        Lt.append(float(x))
    except:
        Lt.append(0)
for y in direct_list:
    try:
        Ld.append(float(y))
    except:
        Ld.append(0)

In [170]:
emis = pd.DataFrame(list(zip(region, Lt, Ld)), columns = ['region_id','totalemission','directemission'])

In [171]:
emis.head(6)

,region_id,totalemission,directemission
0,36061011202,2781.8,0.0
1,36061011202,228.1,0.0
2,36061011202,2253.6,0.0
3,36061011202,1923.4,0.0
4,36061011202,0.0,0.0
5,36061011202,262.3,34.8


In [172]:
emis.shape

(8874, 3)

In [173]:
emis.dropna(inplace=True)

In [175]:
emission = emis.groupby('region_id').agg({'totalemission':'sum'})

In [174]:
demission = emis.groupby('region_id').agg({'directemission':'sum'})

In [252]:
#emission = emission.reset_index()


In [256]:
#demission = demission.reset_index()

In [282]:
#emission['region_id'] = emission['region_id'].astype(str)
#demission['region_id'] = demission['region_id'].astype(str)

In [176]:
emission.head()

,totalemission
region_id,
36005022102,0.0
36005028900,514.8
36047093400,224.2
36061000201,2178.8
36061000202,9633.6


In [177]:
emission.shape

(285, 1)

Data Merging


In [206]:
temp1 = pd.concat([height,elev,landuse,standalone,far,vol,assland,pct,emission,demission], axis=1)

In [207]:
temp1.head()

,heightroof,groundelev,ld01,ld02,ld03,ld04,ld05,ld06,ld07,ld08,...,volume,assessland,assesstot,respct,offpct,retpct,facpct,strpct,totalemission,directemission
region_id,,,,,,,,,,,,,,,,,,,,,
36005022102,72.280000,95.000000,0.0,0.0,4983.0,0.0,0.0,0.0,0.0,0.0,...,8.015595e+05,12.010837,353.732691,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
36005028900,88.653462,11.000000,0.0,0.0,0.0,0.0,0.0,185000.0,0.0,0.0,...,4.531667e+06,13.378378,76.118108,0.000000,0.000000,0.005682,0.0,0.994318,514.8,177.7
36047093400,18.200000,27.000000,0.0,0.0,7550.0,0.0,0.0,0.0,0.0,0.0,...,1.804140e+04,203.841060,440.642384,1.000000,0.000000,0.000000,0.0,0.000000,224.2,136.1
36061000201,44.240000,19.333333,0.0,0.0,0.0,0.0,0.0,0.0,568800.0,178466.0,...,1.029213e+07,19.926639,69.719150,0.000000,0.000000,0.000000,0.0,0.000000,2178.8,978.0
36061000202,118.234759,21.285714,0.0,0.0,352765.0,22750.0,34515.0,0.0,568800.0,71361.0,...,2.064477e+07,27.117210,93.806650,0.655314,0.010778,0.023757,0.0,0.000000,9633.6,2148.1


In [208]:
temp1.shape

(285, 25)

In [209]:
temp1.to_csv('merged0419.csv')

In [57]:
census_tract_path = '/Users/Albert/Jupyter/us_tracts/us_tract.shp'
tracts = gpd.read_file(census_tract_path)

Morning Air

In [178]:
airqm = pd.read_csv('/Users/Albert/Jupyter/ManhattanAirMorning6.csv')

In [181]:
airqm.head()
airqm.shape

(288, 8)

In [186]:
airqm['region_id'] = airqm['region_id'].astype(str)

In [187]:
tempm = airqm.merge(temp1, on= 'region_id')

In [188]:
tempm.head()

,Unnamed: 0,aqi,category,color,pollutant,lat,lng,region_id,heightroof,groundelev,...,volume,assessland,assesstot,respct,offpct,retpct,facpct,strpct,totalemission,directemission
0,1,73,Good air quality,#76CA33,pm10,40.762604,-73.972131,36061011202,295.038846,47.961538,...,1.421244e+08,410476041.0,2.028268e+09,0.083092,0.756442,0.079605,0.0,0.001150,58787.0,2196.0
1,2,76,Good air quality,#69C534,no2,40.728956,-73.971355,36061004400,58.000000,9.000000,...,2.238177e+06,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,1234.7,86.3
2,3,72,Good air quality,#7BCB33,pm10,40.721711,-74.014607,36061003900,118.596244,12.000000,...,9.170093e+07,101331767.0,1.054072e+09,0.624222,0.281913,0.032668,0.0,0.005671,32562.6,12887.6
3,5,71,Good air quality,#7FCD33,pm10,40.753648,-73.974742,36061009200,321.222680,44.818182,...,3.138490e+08,461018701.0,2.437429e+09,0.068227,0.741460,0.028592,0.0,0.000004,111021.8,5119.3
4,6,81,Excellent air quality,#53BD35,no2,40.853979,-73.937385,36061027300,68.850818,221.470588,...,3.786648e+07,18041852.0,1.517801e+08,0.927501,0.003371,0.011865,0.0,0.000000,23743.0,19159.6


In [189]:
tempm.shape

(281, 33)

Correlation

In [192]:
corrs = tempm.drop(['Unnamed: 0', 'category', 'color', 'pollutant','lat','lng','region_id'], axis=1).apply(lambda x: pearsonr(x, tempm['aqi']))
corrs = corrs.reset_index()


In [194]:
corrs[['r', 'p']] = pd.DataFrame(corrs[0].tolist(), 
                                 index=corrs.index)
corrs.drop([0], axis=1, inplace=True)

In [195]:
corrs.head()

,index,r,p
0,aqi,1.000000,0.000000
1,heightroof,-0.146242,0.014138
2,groundelev,0.136972,0.021638
3,ld01,NaN,1.000000
4,ld02,0.144022,0.015688


In [196]:
corrs.shape

(26, 3)

In [197]:
corrs.to_csv('corrsmorning.csv')

In [198]:
corrs.loc[corrs['p'] < .05].sort_values(by='r', ascending=False)[1:20]

,index,r,p
14,alonepct,0.203190,0.000611
4,ld02,0.144022,0.015688
2,groundelev,0.136972,0.021638
19,respct,0.126493,0.034050
1,heightroof,-0.146242,0.014138
16,volume,-0.149103,0.012339
7,ld05,-0.180641,0.002369
21,retpct,-0.198120,0.000839
15,far,-0.211233,0.000363
20,offpct,-0.215562,0.000272
